In [33]:
import torch
from train import train_loop
from data_module import EEGDataModule
from model import ViTransformer, LSTM, ConvLSTM, DeepConvNet, RNN, LSTM, ShallowConvNet
from ATCNet import ATCNet

In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
device = torch.device(0)

In [36]:
from preprocessing import mean_max_subsampling

In [37]:
train_params = {
    "train_epochs": 10,
    "accumulate_grad_batches": 1,
    "test_size": 0.2,
    "random_state": 42,
    "data_dir": "../../Data/",
    "train_batch_size": 64,
    "eval_batch_size": 1
}

In [38]:
dataset = EEGDataModule(args=train_params)

In [39]:
dataset.setup(transform=None)

INFO:data_module:Training data shape: (1692, 22, 1000)
INFO:data_module:Training labels shape: (1692,)


In [40]:
train, valid = dataset.train_dataloader(), dataset.val_dataloader()

INFO:data_module:loaded 1692 train data instances
INFO:data_module:loaded 423 train data instances


In [43]:
model = ATCNet()

In [44]:
from torchinfo import summary
# Only uses outputs of modules.
print(summary(model))

Layer (type:depth-idx)                                  Param #
ATCNet                                                  --
├─Convolution_module: 1-1                               --
│    └─Conv2d: 2-1                                      1,024
│    └─BatchNorm2d: 2-2                                 32
│    └─Conv2d: 2-3                                      704
│    └─BatchNorm2d: 2-4                                 64
│    └─ELU: 2-5                                         --
│    └─AvgPool2d: 2-6                                   --
│    └─AvgPool2d: 2-7                                   --
│    └─Dropout: 2-8                                     --
│    └─Dropout: 2-9                                     --
│    └─Conv2d: 2-10                                     16,384
│    └─BatchNorm2d: 2-11                                64
├─ModuleList: 1-2                                       --
│    └─MultiheadAttention: 2-12                         3,168
│    │    └─NonDynamicallyQuantizableLin

In [25]:
loss_hist, acc_hist, val_loss_hist, val_acc_hist = train_loop(model, train, valid, device, weight_decay=1e-5)

  7%|██                            | 7/100 [00:06<01:27,  1.06it/s, acc=0.423, val_acc=0.253]


KeyboardInterrupt: 

In [12]:
import numpy as np
np.argmax(val_acc_hist)

59

In [13]:
acc_hist[87]

0.9663120567375887